In [ ]:
# THIS IS NOT CONSIDERING SCS AND FCS BASED ON 2019 but ON 2018 so this is ccurrently wrong

In [1]:
import math
import numpy as np
import pandas as pd
from sklearn.metrics import pairwise
import gurobipy as gp
from gurobipy import GRB
from demand_point import DemandPoint
from supply_point import SupplyPoint

demand_history_df = pd.read_csv('./dataset/Demand_History.csv')
limit = len(demand_history_df)
demand_points = []

for i in range(len(demand_history_df[:limit])):
    cur_row = demand_history_df.iloc[i]
    demand_object = DemandPoint(cur_row[0], cur_row[1], cur_row[2], np.array(cur_row[3:]))
    # demand_object.predict_demand()
    demand_points.append(demand_object)

existing_infra_df = pd.read_csv('./dataset/exisiting_EV_infrastructure_2018.csv')
supply_points = []

for i in range(len(existing_infra_df)):
    cur_row = existing_infra_df.iloc[i]
    supply_object = SupplyPoint(cur_row[0], cur_row[1], cur_row[2], cur_row[3], cur_row[4], cur_row[5])
    # supply_object.calculate_max_supply()
    supply_points.append(supply_object)

demand_point_locations = [[point.x, point.y] for point in demand_points]
supply_point_locations = [[point.x, point.y] for point in supply_points]
cost = pairwise.euclidean_distances(demand_point_locations, supply_point_locations)

model = gp.Model('Shell')

"""
Variables
----------

"""

allocation_matrix = {(demand_index, supply_index): model.addVar(vtype = GRB.CONTINUOUS,
                            lb = 0.0,
                            ub = GRB.INFINITY,
                            name = "x_{0}_{1}".format(demand_index, supply_index))
    for demand_index, demand_point in enumerate(demand_points) for supply_index, supply_point in enumerate(supply_points)}

num_fcs = {}
num_scs = {}

num_fcs = {(supply_index) : model.addVar(vtype = GRB.INTEGER,
                lb = supply_point.existing_num_FCS,
                ub = supply_point.total_parking_slots,
                name = "num_fcs_{0}".format(supply_index))
    for supply_index, supply_point in enumerate(supply_points)}

num_scs = {(supply_index) : model.addVar(vtype = GRB.INTEGER,
                lb = supply_point.existing_num_SCS,
                ub = supply_point.total_parking_slots,
                name = "num_scs_{0}".format(supply_index))
    for supply_index, supply_point in enumerate(supply_points)}

"""
Constraints
------------

"""

model.addConstrs(
    (num_fcs[supply_index] + num_scs[supply_index] <= supply_point.total_parking_slots
    for supply_index, supply_point in enumerate(supply_points)),
    name = 'fcs_constr'
)

model.addConstrs(
    (gp.quicksum(allocation_matrix[demand_index, supply_index] 
    for demand_index in range(len(demand_points))) <= 400*num_fcs[supply_index] + 200*num_scs[supply_index] - 10e-3
    
    for supply_index, supply_point in enumerate(supply_points)),
    name = 'constr2'
)

model.addConstrs(
    (gp.quicksum(allocation_matrix[demand_index, supply_index] 
    for supply_index in range(len(supply_points))) <= max(0, demand_point.demand[1] - 10e-3)

    for demand_index, demand_point in enumerate(demand_points)),
    name = 'constr3'
)

# max_possible_supply = sum(supply.max_supply for supply in supply_points)
# model.addConstrs(
#     (gp.quicksum(allocation_matrix[demand_index, supply_index] 
#     for supply_index in range(len(supply_points))
#     for demand_index, demand_point in enumerate(demand_points))) 
#         <=  max_possible_supply
#     ,
#     name = 'constr3'
# )

    
"""
Objective
"""

objective_terms = []
for demand_index, demand_point in enumerate(demand_points):
    for supply_index, supply_point in enumerate(supply_points):
        objective_terms.append(cost[demand_index][supply_index] * allocation_matrix[demand_index, supply_index])

for demand_index, demand_point in enumerate(demand_points):
    objective_terms.append(25 * (
            demand_point.demand[1] - (gp.quicksum(allocation_matrix[demand_index, supply_index] for supply_index in range(len(supply_points))))
        )
    )

for supply_index, supply_point in enumerate(supply_points):
    objective_terms.append(600 * (num_scs[supply_index] + 1.5 * num_fcs[supply_index]))

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-29


In [2]:
model.setObjective(gp.quicksum(objective_terms))
model.ModelSense = gp.GRB.MINIMIZE

In [ ]:
model.computeIIS()
model.write("model.ilp")

In [ ]:
model.feasRelaxS(0, True, False, True)

In [3]:
model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 4296 rows, 409800 columns and 819600 nonzeros
Model fingerprint: 0xcb55fbbd
Variable types: 409600 continuous, 200 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [3e-05, 9e+02]
  Bounds range     [1e+00, 4e+01]
  RHS range        [1e-02, 6e+02]
Found heuristic solution: objective 1.147614e+07
Presolve removed 31 rows and 257871 columns
Presolve time: 0.79s
Presolved: 4265 rows, 151929 columns, 303858 nonzeros
Variable types: 151729 continuous, 200 integer (0 binary)

Root relaxation: objective 3.189353e+06, 7904 iterations, 0.78 seconds (0.25 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 3189353.25    0   23 1.1476e+07 3189353.25  72.2%     -    2s
H    0    

In [ ]:
import matplotlib.pyplot as plt
for demand_index, demand_point in enumerate(demand_points[:100]):
    for supply_index, supply_point in enumerate(supply_points[:100]):
        
#         plt.scatter([supply_point.x], [supply_point.y], color='r')
#         plt.scatter([demand_point.x], [demand_point.y], color='b')
        if allocation_matrix[demand_index, supply_index].X > 0:
            print(demand_index, supply_index, allocation_matrix[demand_index, supply_index].X)
#             plt.text((supply_point.x + demand_point.x) / 2, (supply_point.y + demand_point.y) / 2, '%s'%(allocation_matrix[demand_index, supply_index].X), fontsize = 10)
#             plt.plot([supply_point.x, demand_point.x], [supply_point.y, demand_point.y])
# plt.show()


In [ ]:
allocation_copy = {}

for supply_index, supply_point in enumerate(supply_points):
    for demand_index in range(len(demand_points)): 
        allocation_copy[demand_index, supply_index] = allocation_matrix[demand_index, supply_index].X
        
for supply_index, supply_point in enumerate(supply_points):
    cur = sum([allocation_copy[demand_index, supply_index] for demand_index in range(len(demand_points))])
    max_allowed = supply_point.max_supply

    for demand_index in range(len(demand_points)): 
        if cur <= max_allowed:
            break
        
        cur -= allocation_copy[demand_index, supply_index]
        allocation_copy[demand_index, supply_index] = 0.0

In [ ]:
# Test constraint 1
for supply_index, supply_point in enumerate(supply_points):
    if num_fcs[supply_index].X < 0 or num_scs[supply_index].X < 0 or num_fcs[supply_index].X + num_scs[supply_index].X > supply_point.total_parking_slots:
        print(supply_index, num_fcs[supply_index].X, num_scs[supply_index].X)
    # print(supply_point.total_parking_slots - (num_fcs[supply_index].X + num_scs[supply_index].X))

In [ ]:
# Test constraint 2
for supply_index, supply_point in enumerate(supply_points):
    cur_sum = sum(max(0, allocation_matrix[demand_index, supply_index].X) for demand_index in range(len(demand_points)))
    max_supply = 400 * num_fcs[supply_index].X + 200 * num_scs[supply_index].X
    if cur_sum > max_supply:
        print(supply_index, cur_sum, max_supply, cur_sum - max_supply)
    # print(max_supply - cur_sum)

In [ ]:
# Test constraint 3
for demand_index, demand_point in enumerate(demand_points):
    cur_sum = sum(max(0, allocation_matrix[demand_index, supply_index].X) for supply_index in range(len(supply_points)))
    max_sum = demand_point.demand[1]

    if cur_sum > max_sum:
        print(demand_index, cur_sum, max_sum, cur_sum - max_sum)
    # if max_sum - cur_sum > 0.5:
    #     print(demand_index, max_sum, cur_sum, max_sum - cur_sum)
    # print(demand_index, cur_sum, max_sum, cur_sum - max_sum)

In [ ]:
# Test constraint 4
for demand_index, demand_point in enumerate(demand_points):
    for supply_index, supply_point in enumerate(supply_points):
        if allocation_matrix[demand_index, supply_index].X < 0:
            print(demand_index, supply_index, allocation_matrix[demand_index, supply_index].X )
            

In [ ]:
for supply_index, supply_point in enumerate(supply_points):
    if supply_point.existing_num_FCS > num_fcs[supply_index].X:
        print(supply_index, supply_point.existing_num_FCS, num_fcs[supply_index].X)
    if supply_point.existing_num_SCS > num_scs[supply_index].X:
        print(supply_index, supply_point.existing_num_SCS, num_scs[supply_index].X)

In [7]:
import csv  
with open('shell_notebook.csv', 'a', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    # write the header
    # write multiple rows
    for i in range(len(num_fcs)):
        writer.writerow(['2020', 'FCS', 0, i, int(num_fcs[i].X)])

In [8]:
with open('shell_notebook.csv', 'a', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    for i in range(len(num_scs)):
        writer.writerow(['2020', 'SCS', 0, i, int(num_scs[i].X)])

In [9]:
with open('shell_notebook.csv', 'a', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    for demand_index, demand_point in enumerate(demand_points):
        for supply_index, supply_point in enumerate(supply_points):
            writer.writerow(['2020', 'DS', demand_index, supply_index, max(0, allocation_matrix[demand_index, supply_index].X)])